# Add brunel  jar

In [1]:
%AddJar -magic https://brunelvis.org/jar/spark-kernel-brunel-all-2.3.jar -f

Starting download from https://brunelvis.org/jar/spark-kernel-brunel-all-2.3.jar
Finished download of spark-kernel-brunel-all-2.3.jar


# Analysis of customer churn rate
## 1.1 Load data of customer churn rate

In [2]:
val churnDataRate = spark.read.format("jdbc").options(Map("driver" -> "com.ibm.db2.jcc.DB2Driver",
      "url" -> "jdbc:db2://9.125.72.72:430/LOCDB11", "user" -> "TUSER01",
      "password" -> "C6DESHOP", "dbtable" -> "MLZ.CHURN_RATE")).load()   
churnDataRate.show(5, false)
churnDataRate.count()

+----+-------+------------+----------+
|Year|Quarter|Quarter_Year|Churn_Rate|
+----+-------+------------+----------+
|2014|1      |1Q14        |18.1      |
|2014|2      |2Q14        |18.7      |
|2014|3      |3Q14        |19.3      |
|2014|4      |4Q14        |19.9      |
|2015|1      |1Q15        |20.5      |
+----+-------+------------+----------+
only showing top 5 rows



12

## 1.2 Drawing with the year as the X axis, the churn rate for the vertical 

In [3]:
%%brunel data('churnDataRate') x(QUARTER_YEAR) y(CHURN_RATE) bar tooltip(#all) sort(YEAR:ascending, QUARTER:ascending) tooltip(#all) axes(x:'Time', y:'Churn Rate':grid) 
:: width=800, height=500

## 1.3 Load the detailed customer information

In [4]:
val churnData = spark.read.format("jdbc").options(Map("driver" -> "com.ibm.db2.jcc.DB2Driver",
      "url" -> "jdbc:db2://9.125.72.72:430/LOCDB11", "user" -> "TUSER01",
      "password" -> "C6DESHOP", "dbtable" -> "MLZ.CHURN_CUST_SUM")).load() 
churnData.show(5, false)
churnData.count()

+----------+---+---+---------+----------+-------+--------+-----+--------+------------+-------+----------+---------+-----+----------------+---------+-----------+
|CUST_ID   |SEX|AGE|EDUCATION|INVESTMENT|INCOME |ACTIVITY|CHURN|YRLY_AMT|AVG_DAILY_TX|YRLY_TX|AVG_TX_AMT|NEGTWEETS|STATE|EDUCATION_GROUP |TwitterID|CHURN_LABEL|
+----------+---+---+---------+----------+-------+--------+-----+--------+------------+-------+----------+---------+-----+----------------+---------+-----------+
|1009530860|F  |84 |2        |114368    |3852862|5       |0    |700259.0|0.917808    |335    |2090.32   |3        |TX   |Bachelors degree|0        |0          |
|1009544000|F  |44 |2        |90298     |3849843|1       |0    |726977.0|0.950685    |347    |2095.04   |2        |CA   |Bachelors degree|0        |0          |
|1009534260|F  |23 |2        |94881     |3217364|1       |1    |579084.0|0.920548    |336    |1723.46   |5        |CA   |Bachelors degree|0        |1          |
|1009574010|F  |24 |2        |1120

6001

## 1.4 Drawing with the state as the X axis, the income for the vertical

In [5]:
%%brunel data('churnData') x(STATE) y(INCOME) mean(INCOME) bar tooltip(#all) sort(INCOME:ascending) tooltip(#all) axes(x:'State', y:'Income':grid) 
:: width=800, height=500

## 1.5 Drawing map with the income as the lable

In [6]:
%%brunel data('churnData') map key(STATE) x(STATE) color(INCOME) mean(INCOME) label(STATE) tooltip(#all) :: width=800, height=500

In [7]:

%%brunel data('churnData') 
         x(AGE) y(#count:linear) color(CHURN_LABEL) bin(AGE) interaction(select) stack bar tooltip(#all) filter(CHURN_LABEL) legends(none) |
         x(AVG_DAILY_TX) y(#count:linear) color(CHURN_LABEL) opacity(#selection) bin(AVG_DAILY_TX) stack bar tooltip(#all) axes(x:10:'AVG DAILY TX', y)|
         x(AVG_TX_AMT) y(#count:linear) color(CHURN_LABEL) opacity(#selection) bin(AVG_TX_AMT) stack bar tooltip(#all)  axes(x,y) legends(none) |
        x(INCOME) y(#count:linear) color(CHURN_LABEL) opacity(#selection) bin(INCOME) stack bar tooltip(#all) tooltip(#all) axes(x,y) legends(none)
:: width=800, height=600

In [ ]:
%%brunel data('churnData')
         x(SEX) y(#count:linear) color(CHURN_LABEL) stack bar tooltip(#all) sort(SEX) interaction(select) filter(CHURN_LABEL) axes(x:'GENDER', y) legends(none) |
         x(ACTIVITY) y(#count:linear) color(CHURN_LABEL) stack bar tooltip(#all) sort(ACTIVITY) opacity(#selection)  | 
         x(EDUCATION_GROUP) y(#count:linear) color(CHURN_LABEL) stack bar tooltip(#all) sort(#count) opacity(#selection)  axes(x:'',y) legends(none)
:: width=900, height=600

In [ ]:
%%brunel data('churnData') map key(STATE) x(STATE) color(CHURN) mean(CHURN) label(STATE) tooltip(#all) :: width=800, height=500

# 2. training customer churn classifier 


In [ ]:
//import libraries
import org.apache.spark.ml.feature.{StringIndexer,VectorAssembler}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.tuning.{CrossValidator,ParamGridBuilder}

 ## 2.2 Split the data into training and test data

In [22]:
val Array(training, test) = churnData.randomSplit(Array(0.8, 0.2), seed = 11L)
println("The number of training data is ",training.count())
println("The number of test data is ",test.count())

(The number of training data is ,4884)
(The number of test data is ,1117)


In [35]:
val spamData = spark.read.format("jdbc").options(Map("driver" -> "com.ibm.db2.jcc.DB2Driver",
      "url" -> "jdbc:db2://9.125.72.72:430/LOCDB11", "user" -> "TUSER01",
      "password" -> "C6DESHOP", "dbtable" -> "MLZ.SPAMEMAIL")).load()   
spamData.cache()
spamData.printSchema()
spamData.show(5)
spamData.count()

root
 |-- email_id: long (nullable = true)
 |-- text: string (nullable = true)
 |-- label: integer (nullable = true)

+--------+--------------------+-----+
|email_id|                text|label|
+--------+--------------------+-----+
|       1|One of a kind Mon...|    0|
|      10|Re: What to choos...|    1|
|     100|Strictly Private....|    0|
|    1000|Re: Flash is open...|    1|
|    1001|Re: Alsa/Redhat 8...|    1|
+--------+--------------------+-----+
only showing top 5 rows



2500

## 2.3 Transform the string type to int , and select feature

In [23]:
//Feature definition
val genderIndexer = new StringIndexer().setInputCol("SEX").setOutputCol("gender_code")
val stateIndexer = new StringIndexer().setInputCol("STATE").setOutputCol("state_code")
val labelIndexer = new StringIndexer().setInputCol("CHURN").setOutputCol("label")

val featuresAssembler = new VectorAssembler().setInputCols(Array("AGE", 
                                                         "ACTIVITY", 
                                                         "EDUCATION", 
                                                         "NEGTWEETS" ,
                                                         "INCOME",
                                                         "gender_code",
                                                         "state_code")).setOutputCol("features")

## 2.4 Use the logical regression algorithm, build Pipeline and cross-validate

In [24]:
//Select model automatically in candidate algorithm - Logistic Regression, SVM or Decision Tree?
val lr = new LogisticRegression().setRegParam(0.01).setLabelCol("label").setFeaturesCol("features")
val pipeline =new Pipeline().setStages(Array(labelIndexer, genderIndexer, stateIndexer, featuresAssembler,lr))
val auc_eval = new BinaryClassificationEvaluator()
val grid = new ParamGridBuilder().addGrid(lr.regParam, Array(1e-3, 1e-2)).addGrid(lr.elasticNetParam,Array(0.25, 0.0)).build()
val cross_val = new CrossValidator().setEstimator(pipeline).setEvaluator(auc_eval).setEstimatorParamMaps(grid).setNumFolds(3)

## 2.5 Training model and  predict the test data 

In [25]:
val pipeline_model = cross_val.fit(training)
val trainResult=pipeline_model.transform(test)

In [33]:
trainResult.printSchema()

root
 |-- CUST_ID: integer (nullable = true)
 |-- SEX: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- EDUCATION: integer (nullable = true)
 |-- INVESTMENT: integer (nullable = true)
 |-- INCOME: integer (nullable = true)
 |-- ACTIVITY: integer (nullable = true)
 |-- CHURN: integer (nullable = true)
 |-- YRLY_AMT: double (nullable = true)
 |-- AVG_DAILY_TX: double (nullable = true)
 |-- YRLY_TX: integer (nullable = true)
 |-- AVG_TX_AMT: double (nullable = true)
 |-- NEGTWEETS: integer (nullable = true)
 |-- STATE: string (nullable = true)
 |-- EDUCATION_GROUP: string (nullable = true)
 |-- TwitterID: integer (nullable = true)
 |-- CHURN_LABEL: string (nullable = true)
 |-- label: double (nullable = true)
 |-- gender_code: double (nullable = true)
 |-- state_code: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



## 2.6 Evaluate the model and draw the ROC curve

In [26]:
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
val testingResultScores = trainResult.select("prediction","label").rdd.map(r => (r(0).asInstanceOf[Double], r(1) .asInstanceOf[Double]))
val bc = new BinaryClassificationMetrics(testingResultScores)
val roc = bc.roc

In [34]:
testingResultScores

MapPartitionsRDD[1574] at map at <console>:53

In [32]:
print( bc.roc())

UnionRDD[1599] at UnionRDD at BinaryClassificationMetrics.scala:89

In [27]:
val rocDF = spark.createDataFrame(roc).
                    withColumnRenamed("_1", "FPR").
                    withColumnRenamed("_2", "TPR")
rocDF.show()

+--------------------+------------------+
|                 FPR|               TPR|
+--------------------+------------------+
|                 0.0|               0.0|
|0.004813477737665...|0.9545454545454546|
|                 1.0|               1.0|
|                 1.0|               1.0|
+--------------------+------------------+



In [28]:
%%brunel data('rocDF') x(FPR) y(TPR) line tooltip(#all) axes(x:'False Positive Rate':grid, y:'True Positive Rate':grid)

In [29]:
case class Person(AGE:Int,ACTIVITY:Int,EDUCATION:Int,NEGTWEETS:Int,INCOME:Int,SEX:String,STATE:String)
val dataFrame=Seq(Person(40,1,3,4,200000,"M","TX"),Person(40,1,3,8,200000,"M","OR"))
val df = spark.createDataFrame(dataFrame)
df.show()
pipeline_model.transform(df).show()

+---+--------+---------+---------+------+---+-----+
|AGE|ACTIVITY|EDUCATION|NEGTWEETS|INCOME|SEX|STATE|
+---+--------+---------+---------+------+---+-----+
| 40|       1|        3|        4|200000|  M|   TX|
| 40|       1|        3|        8|200000|  M|   OR|
+---+--------+---------+---------+------+---+-----+

+---+--------+---------+---------+------+---+-----+-----------+----------+--------------------+--------------------+--------------------+----------+
|AGE|ACTIVITY|EDUCATION|NEGTWEETS|INCOME|SEX|STATE|gender_code|state_code|            features|       rawPrediction|         probability|prediction|
+---+--------+---------+---------+------+---+-----+-----------+----------+--------------------+--------------------+--------------------+----------+
| 40|       1|        3|        4|200000|  M|   TX|        0.0|      27.0|[40.0,1.0,3.0,4.0...|[4.52523954313736...|[0.98928395807943...|       0.0|
| 40|       1|        3|        8|200000|  M|   OR|        0.0|       5.0|[40.0,1.0,3.0,8.0